In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { readFileSync, writeFileSync } from "fs";
import { buildParser } from "@lezer/generator";
import { Tree, TreeCursor } from "@lezer/common";
import { LRParser } from "@lezer/lr";
import { AST, cstToAST, ast2dot, ParserConfig, NumNode, VarNode, ExprStmtNode, BlockNode, AssignNode, BinaryExpr, WhileNode, IfNode, CallNode, NilNode, getVarName, getBlockStmts, ASTNode } from "./AST2Dot";
import { RecursiveMap } from "recursive-set";
import { instance } from "@viz-js/viz";
const viz = await instance();

# Building a Complete Interpreter with Lezer

In this notebook, we build a fully functional interpreter for a simple `C`-like language.
We will implement the Scanner and Parser using **Lezer**, transform the result into a clean **AST**, and finally write an **Interpreter** that executes the code.

## The Language Specification

Our target language supports arithmetic, variables, control flow (`if`, `while`), and function calls.
Formally, the grammar is defined as follows:

```ebnf
program
    : 𝜆 
    | stmnt program
    
stmnt 
    : IF '(' bool_expr ')' stmnt                 
    | WHILE '(' bool_expr ')' stmnt
    | '{' program '}' 
    | IDENTIFIER ':=' expr ';'  
    | expr ';'       

bool_expr 
    : expr '==' expr     
    | expr '!=' expr     
    | expr '<=' expr     
    | expr '>=' expr     
    | expr '<'  expr      
    | expr '>'  expr     
 
expr: expr '+' product                 
    | expr '-' product
    | product
              
product
    : product '*' factor               
    | product '/' factor
    | product '%' factor 
    | factor

factor
    : '(' expr ')' 
    | NUMBER
    | IDENTIFIER
    | IDENTIFIER '(' expr_list ')'

expr_list
    : 𝜆 
    | ne_expr_list

ne_expr_list
    : expr
    | expr ',' ne_expr_list
```

## Defining the Lexical Grammar (Tokens)

The **Lexical Grammar** defines how raw text is split into tokens. We will define the content of the `@tokens` block in several parts and assemble them at the end.

### Whitespace and Comments

**Whitespace**: We define `space` to match tabs, newlines, and carriage returns.

**Block Comments (`/* ... */`)**:
Recognizing C-style block comments is tricky because they have a multi-character ending `*/`. Regexes are sometimes insufficient or hard to read for this.
Instead, we define a small **State Machine**:

1.  **`BlockComment`** (Entry): Matches the start `/*` and transitions to the state `blockCommentRest`.
2.  **`blockCommentRest`** (Content State): This is the **body** of the comment. It consumes characters that are **not** `*`. If it sees a `*`, it transitions to the "Potential End State" (`blockCommentAfterStar`).
3.  **`blockCommentAfterStar`** (Potential End State): We just saw a `*`. We check the next character to decide if we are really done:
    * If it is `/`: We found the end `*/`. The token is complete.
    * If it is `*`: We might have `/**`, so we stay in this state (waiting for a slash or non-star).
    * Anything else: It was just a standalone `*` inside the text (like in `2 * 3`), so we go back to the "Content State".

In [ ]:
const tokensComments = `
    space { $[ \\t\\n\\r]+ }
    LineComment { "//" ![\\n]* }
    BlockComment { "/*" blockCommentRest }
    blockCommentRest { ![*] blockCommentRest | "*" blockCommentAfterStar }
    blockCommentAfterStar { "/" | "*" blockCommentAfterStar | ![/*] blockCommentRest }
`;

### Keywords and Identifiers

This section handles names.
* **Identifiers**: Variable names.

In [ ]:
const tokensIdent = `
    Identifier { $[a-zA-Z] $[a-zA-Z0-9_]* }
`;

### Literals and Operators

Here we define the static symbols of our language.
* **Numbers**: Note the pattern `0 | $[1-9] $[0-9]*`. This strict rule prevents numbers with leading zeros (like `012`), ensuring only valid integers are parsed.
* **Operators**: We map literal characters (like `+`) to token names (like `OpPlus`).

In [ ]:
const tokensOps = `
    Number { "0" | $[1-9] $[0-9]* }

    OpAssign { ":=" }
    OpEq { "==" } OpNe { "!=" }
    OpLe { "<=" } OpGe { ">=" }
    OpLt { "<" }  OpGt { ">" }
    
    OpPlus { "+" } OpMinus { "-" } 
    OpTimes { "*" } OpDivide { "/" } OpModulo { "%" }
    
    LParen { "(" } RParen { ")" }
    LBrace { "{" } RBrace { "}" }
    Semi { ";" } Comma { "," }
`;

### Conflict Resolution & Assembly

Now we assemble the `@tokens` block.
Crucially, we define **Lexical Precedence** here to solve the ambiguities mentioned above:

1.  `KwIf` > `Identifier`: Ensures "if" is treated as a keyword.
2.  `LineComment` > `OpDivide`: Ensures `//` is a comment, not two division operators.
3.  `@skip`: Tells the parser to completely ignore spaces and comments in the resulting syntax tree.

In [ ]:
const grammarTokens = `
  @tokens {
    ${tokensComments}
    ${tokensIdent}
    ${tokensOps}
    @precedence { LineComment, BlockComment, OpDivide }
  }
  @skip { space | LineComment | BlockComment }
`;

## Operator Precedence

In arithmetic expressions, ambiguity arises. For `1 + 2 * 3`, two trees are possible: `(1+2)*3` or `1+(2*3)`.

We solve this by defining **Precedence Levels**.
* **`times`**: Highest priority (binds tightest).
* **`plus`**: Medium priority.
* **`compare`**: Lowest priority.

The `@left` annotation indicates **Left Associativity**. `1 - 2 - 3` is parsed as `(1 - 2) - 3`, not `1 - (2 - 3)`.

In [ ]:
const grammarPrecedence : string = `
    @precedence {
        times @left,
        plus @left,
        compare @left
    }
`;

## Syntactic Grammar (Rules)

We now define the structure of our language. We split the rules into logical blocks and concatenate them at the end.

### The Entry Point
The **`Program`** is the start symbol of our grammar.
It is defined simply as a sequence of zero or more `statement` nodes.

In [ ]:
const ruleTop : string = `
    @top Program { statement* }
`;

In [ ]:
const ruleKeywords = `
    KwIf     { @specialize<Identifier, "if"> }
    KwWhile  { @specialize<Identifier, "while"> }
`;

### Statements

A **`statement`** represents a standalone instruction. It acts as a wrapper for various specific statement types.
* **Control Flow**: `IfStatement` and `WhileStatement` define the structure for branching and looping. Note how they recursively use `statement` for their bodies.
* **Block**: A group of statements enclosed in curly braces `{ ... }`.
* **Assignment**: Assigns an expression (`Expr`) to an `Identifier`.
* **Expression Statement**: A standalone expression followed by a semicolon (e.g., a function call `print(x);`).

In [ ]:
const ruleStatements : string = `
    statement {
        IfStatement { KwIf LParen BoolExpr RParen statement } |
        WhileStatement { KwWhile LParen BoolExpr RParen statement } |
        Block { LBrace statement* RBrace } |
        Assignment { Identifier OpAssign Expr Semi } |
        ExprStatement { Expr Semi }
    }
  
    BoolExpr {
        Compare { Expr (OpEq | OpNe | OpLe | OpGe | OpLt | OpGt) Expr }
    }
`;

### Expressions

An **`Expr`** (Expression) represents a value computation.
It serves as a "dispatcher" rule that delegates to:
1.  **`BinaryExpression`**: Arithmetic or Logic.
2.  **`FunctionCall`**: Calling a function with arguments.
3.  **`Atom`**: The basic building blocks (numbers, variables).

In [ ]:
const ruleExpr = `
    Expr {
        BinaryExpr { Expr (OpPlus | OpMinus) Product } |
        Product
    }
`;

In [ ]:
const ruleProduct = `
    Product {
        BinaryExpr { Product (OpTimes | OpDivide | OpModulo) Factor } |
        Factor
    }
`;

In [ ]:
const ruleFactor = `
    Factor {
        ParenExpr { LParen Expr RParen } |
        Number |
        Identifier |
        Call { Identifier LParen ArgList? RParen }
    }

    ArgList {
        Expr (Comma Expr)*
    }
`;

### Binary Expressions & Precedence Handling

This is where the **Precedence** defined earlier is applied.
Instead of writing a complex recursive grammar (like `Expr -> Term -> Factor`), Lezer allows us to write a flat rule and apply **Precedence Markers** (`!times`, `!plus`, `!compare`).

* `Expr !times (OpTimes | ...) Expr`: This tells Lezer that this rule belongs to the `times` precedence group.
* Because `times` was defined above `plus`, Lezer knows that `*` binds tighter than `+`.

In [ ]:
const ruleBinary : string = `
    BinaryExpression {
        Expr !compare (OpEq | OpNe | OpLe | OpGe | OpLt | OpGt) Expr |
        Expr !plus (OpPlus | OpMinus) Expr |
        Expr !times (OpTimes | OpDivide | OpModulo) Expr
    }
`;

### Argument Lists and Atoms

* **`ArgList`**: Defines a comma-separated list of expressions. Used in function calls.
* **`Atom`**: The base case of our recursion.
    * `Number`: A raw numeric value.
    * `Identifier`: A variable name.
    * `LParen Expr RParen`: Parentheses allow us to override precedence manually (e.g., `(1+2)*3`).

In [ ]:
const ruleAtoms : string = `
    ArgList {
        Expr (Comma Expr)*
    }

    Atom {
        Number |
        Identifier |
        LParen Expr RParen
    }
`;

### Building the Parser

Finally, we concatenate all string parts to form the complete grammar definition and generate the parser.

In [ ]:
const grammarString = 
    grammarTokens + 
    ruleTop + 
    ruleKeywords + 
    ruleStatements + 
    ruleExpr + 
    ruleProduct + 
    ruleFactor;
grammarString;

In [ ]:
const parser : LRParser = buildParser(grammarString);
"Parser generated successfully.";

## Testing the Scanner (Token Verification)

Since Lezer combines tokenization and parsing, we do not get a flat list of tokens automatically. However, we can simulate a scanner check by extracting the leaves of the resulting **Concrete Syntax Tree (CST)**.

**Formal Definition:**

Let $S$ be the input string and $T$ be the Syntax Tree generated by the parser.
We define the sequence of Tokens $\mathcal{T}$ as the ordered sequence of all leaf nodes in $T$.

A node $n \in T$ is considered a leaf if and only if it has no children:
$$\text{isLeaf}(n) \iff \text{degree}^{+}(n) = 0$$

The function `testScanner` performs a Depth-First Traversal over $T$. For every node $n$ visited by the cursor $C$, we execute the following logic:

$$
\text{Output}(n) =
\begin{cases}
    \texttt{print}(n.\text{type}, S[n.\text{from} \dots n.\text{to}]) & \text{if } \neg C.\text{firstChild}() \\
    \text{continue} & \text{otherwise}
\end{cases}
$$

In [ ]:
import { Tree, TreeCursor } from "@lezer/common";

function testScanner(fileName: string): void {
    const input: string = readFileSync(fileName, "utf8");
    
    console.log(`--- Scanning ${fileName} ---`);
    console.log(input);
    console.log("Tokens:");
    
    const tree: Tree = parser.parse(input);
    const cursor: TreeCursor = tree.cursor();
    
    do {
        if (!cursor.firstChild()) {
            const tokenText = input.slice(cursor.from, cursor.to);
            const safeText = tokenText.replace(/\n/g, "\\n");
            console.log(`[${cursor.name}]`.padEnd(15) + `: ${safeText}`);
        }
    } while (cursor.next());
}

In [ ]:
testScanner('sum.sl');

In [ ]:
testScanner('factorial.sl');

## The CST-to-AST Transformation

Lezer generates a **Concrete Syntax Tree (CST)**. This tree contains every single detail of the source code, including whitespace, comments, parentheses, and semicolons. While perfect for syntax highlighting, this structure is too noisy for interpretation.

We need to transform this CST into an **Abstract Syntax Tree (AST)**. An AST focuses purely on the logical structure of the program (e.g., "This is an assignment") rather than the syntactic sugar (e.g., "There is a semicolon here").

### The Configuration Object (`ParserConfig`)

To perform this transformation generically, we define an **`ParserConfig`** object. This object serves as a blueprint for the mapper. It has three distinct purposes:

1.  **Noise Reduction (`ignore`)**:
    We define a set of token names that carry no semantic meaning for the interpreter. Tokens like `LParen` (`(`), `Semi` (`;`), or keywords like `KwIf` are strictly syntactic. They guide the parser but are irrelevant for the logical tree. The mapper automatically discards these nodes.

2.  **Leaf Extraction (`treatAsLiteral`)**:
    Some nodes, like **Operators** or **Identifiers**, are effectively "leaves" in our logic. We do not need to traverse inside them; we simply want their text content (e.g., the string `"x"` or `"+"`). We use a Regular Expression to identify these nodes and extract their raw text immediately.

3.  **Structural Transformation (`rules`)**:
    This is the core logic. For every complex node (like `BinaryExpression` or `Assignment`), we define a specific transformation function.
    * **Input**: A list of already processed children (sanitized of noise).
    * **Output**: A strictly typed `AST` object (e.g., `{ kind: 'Assignment', ... }`).

By separating the transformation logic (the generic walker) from the specific rules (the `ASTConfig`), we achieve a modular and type-safe design.

In [ ]:
const config: ParserConfig = {
    ignore: new Set([
        "LParen", "RParen",
        "LBrace", "RBrace",
        "Semi", "Comma",
        "OpAssign"
    ]),
    rules: {
        "Number": (_, text) => new NumNode(parseInt(text)),
        "Identifier": (_, text) => new VarNode(text),
        "OpPlus": () => new VarNode("+"), "OpMinus": () => new VarNode("-"),
        "OpTimes": () => new VarNode("*"), "OpDivide": () => new VarNode("/"), 
        "OpModulo": () => new VarNode("%"),
        "OpEq": () => new VarNode("=="), "OpNe": () => new VarNode("!="),
        "OpLe": () => new VarNode("<="), "OpGe": () => new VarNode(">="),
        "OpLt": () => new VarNode("<"),  "OpGt": () => new VarNode(">"),
        "Program": (children) => new BlockNode(children),
        "Block": (children) => new BlockNode(children),
        "Assignment": (children) => {
            const id = children[0];
            if (id instanceof VarNode) return new AssignNode(id.name, children[1]);
            throw new Error("Invalid Assignment");
        },
        "KwIf": () => new VarNode("if"),
        "KwWhile": () => new VarNode("while"),
        "IfStatement": (children) => {
             const real = children.filter(c => !(c instanceof VarNode && (c.name === "if" || c.name === "while")));
             return new IfNode(real[0], real[1]);
        },
        "WhileStatement": (children) => {
             const real = children.filter(c => !(c instanceof VarNode && c.name === "while"));
             return new WhileNode(real[0], real[1]);
        },
        "ExprStatement": (children) => new ExprStmtNode(children[0]),
        "Expr": (children) => children[0],
        "Product": (children) => children[0],
        "BoolExpr": (children) => children[0],
        "ParenExpr": (children) => children[0],
        "BinaryExpr": (children) => {
             const op = children[1];
             if (op instanceof VarNode) {
                 return new BinaryExpr(children[0], op.name, children[2]);
             }
             throw new Error(`Invalid BinaryExpr. Got children: ${children.map(c => c.constructor.name)}`);
        },
        "Compare": (children) => {
             const op = children[1];
             if (op instanceof VarNode) {
                 return new BinaryExpr(children[0], op.name, children[2]);
             }
             throw new Error("Invalid Compare");
        },
        "Call": (children) => {
            const id = children[0];
            let args: AST[] = [];
            if (children.length > 1 && children[1] instanceof BlockNode) {
                args = (children[1]).statements;
            }
            if (id instanceof VarNode) return new CallNode(id.name, args);
            throw new Error("Invalid Call");
        },
        "ArgList": (children) => new BlockNode(children)
    }
};

function parse(fileName: string): AST {
    const source = readFileSync(fileName, "utf8");
    const tree = parser.parse(source);
    return cstToAST(tree.cursor(), source, config);
}

### Visualizing the AST

To verify that our configuration correctly transforms the CST into the intended AST structure, we utilize our `AST2Dot` library. This renders the tree structure, showing the **Program** root, the statements, and the nested expressions.

In [ ]:
const astSum = parse("sum.sl");
const dotSum = ast2dot(astSum);
display.html(viz.renderString(dotSum, { format: "svg" }));

In [ ]:
const astFact = parse("factorial.sl");
const dotFact = ast2dot(astFact);
display.html(viz.renderString(dotFact, { format: "svg" }));

## The Interpreter

The interpreter breathes life into our **Abstract Syntax Tree (AST)**. It recursively traverses the tree structure and performs the operations described by the nodes.

We divide the implementation into three specialized functions, mirroring the structure of our AST:

1.  **`execute`**: Handles **Statements** (side effects). It modifies the program state but returns nothing (`void`).
2.  **`evaluate`**: Handles **Arithmetic Expressions**. It calculates and returns a `number`.
3.  **`evaluateBool`**: Handles **Conditions**. It returns a `boolean` to decide control flow paths.

### State Management

Instead of plain JavaScript objects (which rely on reference identity), we use `RecursiveMap` from the `recursive-set` library. This ensures that our environment itself has **Value Semantics**.

* **Keys**: Variable names (`string`).
* **Values**: The stored numbers (`number`).
* **API**: We use `.get(key)` to retrieve values and `.set(key, val)` to update the state.

In [ ]:
type Variables = RecursiveMap<string, number>;
let inputStream: string[] = [];
let execute: (node: AST, values: Variables) => void;
let evaluate: (node: AST, values: Variables) => number;
let evaluateBool: (node: AST, values: Variables) => boolean;

### Executing Statements (`execute`)

The `execute` function serves as the control center. It accepts an `AST` node and the current `values` (memory).

**The Logic:**
Instead of a switch-statement on strings, we now use **Type Guards** (`instanceof`) to determine the specific class of the node. This allows TypeScript to narrow down the type safely.

* **`BlockNode`**: Iterates through its `statements` array and recursively calls `execute` for each one.
* **`AssignNode`**: Computes the right-hand side using `evaluate` and updates the `values` map.
* **`IfNode` / `WhileNode`**: Control flow structures relying on `evaluateBool`.

In [ ]:
execute = (node: AST, values: Variables): void => {    
    if (node instanceof BlockNode) {
        for (const stmt of node.statements) {
            execute(stmt, values);
        }
        return;
    }
    if (node instanceof AssignNode) {
        const result = evaluate(node.expr, values);
        values.set(node.id, result); 
        return;
    }
    if (node instanceof ExprStmtNode) {
        evaluate(node.expr, values);
        return;
    }
    if (node instanceof IfNode) {
        if (evaluateBool(node.cond, values)) {
            execute(node.thenB, values);
        } else if (!(node.elseB instanceof NilNode)) {
            execute(node.elseB, values);
        }
        return;
    }
    if (node instanceof WhileNode) {
        while (evaluateBool(node.cond, values)) {
            execute(node.body, values);
        }
        return;
    }
    if (node instanceof NilNode) return;
    throw new Error(`Runtime Error: Cannot execute node: ${node}`);
}

### Evaluating Conditions (`evaluateBool`)

**The Logic:**
1.  **Comparisons (`BinaryExpr`)**: We check the operator. If it is a relational operator (e.g. `<`), we evaluate both operands and return the boolean result.
2.  **C-Style Truthiness**: If the node is not an explicit comparison (e.g. just a number or an arithmetic term), we follow the convention: `0` is `false`, everything else is `true`.

In [ ]:
evaluateBool = (node: AST, values: Variables): boolean => {
    if (node instanceof BinaryExpr) {
        const l = evaluate(node.left, values);
        const r = evaluate(node.right, values);
        switch (node.op) {
            case '==': return l === r;
            case '!=': return l !== r;
            case '<':  return l < r;
            case '>':  return l > r;
            case '<=': return l <= r;
            case '>=': return l >= r;
        }
    }
    const val = evaluate(node, values);
    return val !== 0;
};

### Evaluating Expressions (`evaluate`)

This function computes the actual numerical values.

**The Logic:**
* **Leaves**:
    * **Numbers**: Returned "as is".
    * **Variables (Strings)**: Looked up in the `values` map. We throw an error if the variable is undefined.
* **Binary Expressions**: We recursively evaluate `left` and `right` operands and apply the math operator. Note that `/` performs **integer division** (`Math.floor`) to keep things simple.
* **Function Calls**:
    * **`print`**: Evaluates its argument, logs it to the console (simulating `stdout`), and returns `0`.
    * **`read`**: Simulates reading from `stdin`. It takes the next value from the global `inputStream` queue. If the queue is empty, it throws a Runtime Error.

In [ ]:
evaluate = (node: AST, values: Variables): number => {
    if (node instanceof NumNode) {
        return node.value;
    }
    if (node instanceof VarNode) {
        const val = values.get(node.name);
        if (val === undefined) {
            throw new Error(`Runtime Error: Undefined variable '${node.name}'`);
        }
        return val;
    }
    if (node instanceof BinaryExpr) {
        const l = evaluate(node.left, values);
        const r = evaluate(node.right, values);
        switch (node.op) {
            case '+': return l + r;
            case '-': return l - r;
            case '*': return l * r;
            case '/': return Math.floor(l / r); 
            case '%': return l % r;
            default: throw new Error(`Runtime Error: Operator '${node.op}' yields boolean, expected number.`);
        }
    }
    if (node instanceof CallNode) {
        if (node.fn === "print") {
            const val = node.args.length > 0 ? evaluate(node.args[0], values) : 0;
            console.log(">> STDOUT:", val);
            return 0;
        }
        if (node.fn === "read") {
            const input = inputStream.shift();
            if (input === undefined) {
                throw new Error("Runtime Error: 'read()' called but Input Stream is empty!");
            }
            console.log(`<< STDIN: Read '${input}'`);
            const num = parseInt(input, 10);
            if (isNaN(num)) {
                throw new Error(`Runtime Error: Input '${input}' is not a number.`);
            }
            return num;
        }

        throw new Error(`Runtime Error: Unknown function '${node.fn}'`);
    }
    throw new Error(`Runtime Error: Cannot evaluate expression from node type: ${node.constructor.name}`);
};


## Execution

Finally, we define a `main` function that puts everything together:
1.  Read the source file.
2.  Parse it into an AST.
3.  Initialize an empty memory.
4.  Execute the AST.

In [ ]:
function main(fileName: string, inputs: string[] = []) {
    console.log(`\n--- Executing ${fileName} with inputs [${inputs.join(", ")}] ---`);
    inputStream = [...inputs]; 
    
    try {
        const ast = parse(fileName);
        const values: Variables = new RecursiveMap();
        
        execute(ast, values);
        
        console.log("Final Memory State:", values);
    } catch (e) {
        if (e instanceof Error) {
            console.error(e.message);
        } else {
            console.error("Unbekannter Fehler:", String(e));
        }
    }
}

In [ ]:
main('sum.sl', ["5"]);

In [ ]:
main('sum.sl');

In [ ]:
main('sum.sl', ["6"]);

In [ ]:
main('factorial.sl', ["10"]);

In [ ]:
main('factorial.sl', ["hello"]);